# Import Statements

In [228]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import os
import wget
import time
import pandas as pd

# Disabling Google Location Notification

In [229]:
options1 = Options()
options1.add_argument("--disable-notifications")

# Setting Driver Path And Initializing Driver

In [230]:
driver_path = 'C:/Users/sanje/Chrome Drivers/chromedriver.exe'
driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options1)

C:\Users\sanje\AppData\Local\Temp/ipykernel_1180/226995219.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options1)
C:\Users\sanje\AppData\Local\Temp/ipykernel_1180/226995219.py:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options1)


# Getting The Designated Website

In [231]:
link = 'https://www.carwale.com/used/cars-for-sale/#sc=-1&so=-1&city=105&pc=105'

In [232]:
driver.get(link)

# Loading the whole page in one shot

In [233]:
SCROLL_PAUSE_TIME = 0.5

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Extracting Whole Chunk Of Data

In [234]:

#     The chunk of data is (year of manufacture, name of the model, price of the car, price of
#     the car, kms driven, fuel type, fuel type, city of residence of the owner)
    
#     There are multiple fuel type values and price of the car because of the formatting
#     of the website.
    
#     In the next sections we try to extract each of them out and create a list of final
#     cars.
    

In [235]:
load = driver.find_elements_by_css_selector('.card-detail-block__data-table span')

C:\Users\sanje\AppData\Local\Temp/ipykernel_1180/964615350.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  load = driver.find_elements_by_css_selector('.card-detail-block__data-table span')


In [236]:
len(load)

1966

# Creating the list of years that maybe present

In [237]:
lsofyears = [] 
for i in range(1990, 2025):
    lsofyears.append(str(i))

# Creating Each Car Data

In [238]:
popo= []
c=0
final= []
for i in range(0,len(load),1):
    if(load[i].text in lsofyears):
        c+=1
        if(c == 1):
            popo.append(load[i].text)
            clicks.append(load[i+1])
        elif(c == 2):
            c = 1
            final.append(popo)
            popo = []
            popo.append(load[i].text)
            clicks.append(load[i+1])
    else:
        popo.append(load[i].text)
            

In [239]:
len(final)

245

In [240]:
final[0:2]

[['2016',
  'Maruti Suzuki Baleno Zeta 1.3',
  '₹ 6.8 Lakh',
  '6.8 Lakh',
  '93,525 km',
  'Diesel',
  'Diesel',
  'Hyderabad'],
 ['2017',
  'Maruti Suzuki Wagon R 1.0 VXI AMT',
  '₹ 5.2 Lakh',
  '5.2 Lakh',
  '40,904 km',
  'Petrol',
  'Petrol',
  'Hyderabad']]

# Finding Data Abberations

In [241]:
for i in final:
    if(len(i) != 8):
        print(i, len(i))

['2012', 'Maruti Suzuki Wagon R 1.0 LXi CNG', '₹ 3.2 Lakh', '3.2 Lakh', '65,000 km', 'CNG + 1', 'Hyderabad'] 7
['2012', 'Maruti Suzuki Wagon R 1.0 LXi CNG', '₹ 3.2 Lakh', '3.2 Lakh', '65,000 km', 'CNG + 1', 'Hyderabad'] 7


# Removing Redundant Data From List

In [242]:
for i in final :
    if(len(i) == 8):
        i.pop(2)
        i.pop(4)
    elif(len(i) == 7):
        i.pop(2)   

# Creating List Of Individual Columns

In [243]:
year = []
model =[]
price = []
kms = []
fuel = []
city= []

In [244]:
for i in final:
    year.append(i[0])
    model.append(i[1])
    price.append(i[2])
    kms.append(i[3])
    fuel.append(i[4])
    city.append(i[5])

# Converting Str To Float For kms And price

In [245]:
temp_price = []
for i in price:
    if(len(i.split(' '))>1):
        d= i.split(' ')[1]
    if(len(i.split(' ')) == 1):  #thousands case
        var = i.split(' ')[0]
        var = var.replace(',','')
        temp_price.append(float(var))
    elif(d =='Lakh'):  #lakhs case
        temp_price.append(float(i.split(' ')[0])*100000)
    elif(d == 'Crore'):   #crore case
        temp_price.append(float(i.split(' ')[0])*10000000)

In [246]:
price = temp_price

In [247]:
temp_kms = []
for i in kms:
    i = i.replace(',','')
    d= i.split(' ')[1]
    if(d =='km'):
        temp_kms.append(float(i.split(' ')[0]))

In [248]:
kms  = temp_kms

In [249]:
len(price)

245

# Creating Brand Column

In [250]:
brand = []
for i in model:
    var = i.split(' ')[0]
    if(var == 'Land'):
        brand.append('Land Rover')
    elif(var == 'Force'):
        brand.append('Force Motors')
    elif(var == 'Maruti'):
        brand.append('Maruti Suzuki')
    else:
        brand.append(var)

In [251]:
len(brand)

245

# Creating model name 

In [252]:
model_name = []
for i in model:
    var = i.split(' ')
    if(var[0] == 'Land' and var[2] == 'Range'):
        model_name.append(var[4])
    elif(var[0] == 'Land' and var[2] != 'Range'):
        model_name.append(var[2])
    elif(var[0] == 'Force'):
        model_name.append(var[2])
    elif(var[0] == 'Maruti'):
        model_name.append(var[2])
    else:
        model_name.append(var[1])

In [253]:
# model_name

# Creating Dictionary And Dataframe

In [254]:
dict = {'year':year,'brand':brand,'full_model_name':model, 'model_name':model_name, 'price':price, 'distance_travelled(kms)': kms, 'fuel_type' : fuel, 'city':city}

In [255]:
df = pd.DataFrame(dict)

In [256]:
df

,year,brand,full_model_name,model_name,price,distance_travelled(kms),fuel_type,city
0,2016,Maruti Suzuki,Maruti Suzuki Baleno Zeta 1.3,Baleno,680000.0,93525.0,Diesel,Hyderabad
1,2017,Maruti Suzuki,Maruti Suzuki Wagon R 1.0 VXI AMT,Wagon,520000.0,40904.0,Petrol,Hyderabad
2,2004,Toyota,Toyota Qualis GS G1,Qualis,240000.0,209616.0,Diesel,Hyderabad
3,2014,Maruti Suzuki,Maruti Suzuki Ertiga ZDi,Ertiga,735000.0,43294.0,Diesel,Hyderabad
4,2013,Mercedes-Benz,Mercedes-Benz E-Class E250 CDI Avantgarde,E-Class,1850000.0,89000.0,Diesel,Hyderabad
...,...,...,...,...,...,...,...,...
240,2010,Ford,Ford Endeavour 3.0L 4x4 AT,Endeavour,685000.0,115000.0,Diesel,Hyderabad
241,2020,Hyundai,Hyundai Creta SX (O) 1.5 Diesel,Creta,1890000.0,23906.0,Diesel,Hyderabad
242,2017,Mahindra,Mahindra Thar CRDe 4x4 AC1,Thar,875000.0,76000.0,Diesel,Hyderabad
243,2018,Hyundai,Hyundai Grand i10 Asta 1.2 Kappa VTVT [2013-2016],Grand,625000.0,43000.0,Petrol,Hyderabad


# Saving Dataframe To CSV

In [257]:
df.to_csv('UsedCars_Hyderabad.csv')